In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc # Garbage Collector
import os
import sys
path = ""

## LIGHTGBM AISTUDIO BASELINE 0.44

In [ ]:
data = pd.read_csv('data/processed_data.csv')

Début du script de baseline LightGBM...
Préparation finale des données...
Séparation des jeux d'entraînement et de test...
Création du jeu de validation...
Jeu d'entraînement partiel : 2972640 lignes
Jeu de validation : 28512 lignes

Entraînement du modèle LightGBM avec suivi de la progression...
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.796793
[200]	valid_0's rmse: 0.519378
[300]	valid_0's rmse: 0.495381
[400]	valid_0's rmse: 0.491092
[500]	valid_0's rmse: 0.489253
[600]	valid_0's rmse: 0.487952
[700]	valid_0's rmse: 0.487843
[800]	valid_0's rmse: 0.487726
[900]	valid_0's rmse: 0.48746
[1000]	valid_0's rmse: 0.486957
[1100]	valid_0's rmse: 0.486689
Early stopping, best iteration is:
[1093]	valid_0's rmse: 0.486689

Génération des prédictions sur le jeu de test...
Meilleure itération trouvée : 1093

Fichier 'submission_baseline_lgbm_v2.csv' créé avec succès !
              id     sales
3001152  3000888  4.475289
3002934  3002670  4.197102
300

In [ ]:

# ==============================================================================
# ÉTAPE 1: PRÉPARATION FINALE POUR LE MODÈLE
# ==============================================================================

print("Préparation finale des données...")

# !! CORRECTION : On ne supprime pas la colonne 'date' tout de suite !!
if 'transactions' in data.columns:
    data.drop(columns=['transactions'], inplace=True)

# Conversion des colonnes catégorielles
categorical_features = [
    "store_nbr", "family", "city", "state", "type", "cluster",
    "is_holiday", "dayofweek", "month"
]
for col in categorical_features:
    if col in data.columns:
        data[col] = data[col].astype('category')

# ==============================================================================
# ÉTAPE 2: SÉPARATION DES DONNÉES EN ENTRAÎNEMENT ET TEST
# ==============================================================================
print("Séparation des jeux d'entraînement et de test...")
train_df = data[data['is_train'] == 1].copy()
test_df = data[data['is_train'] == 0].copy()

# ==============================================================================
# ÉTAPE 3 BIS: CRÉATION D'UN JEU DE VALIDATION (MÉTHODE CORRIGÉE)
# ==============================================================================
print("Création du jeu de validation...")

# On utilise la colonne 'date' QUI EXISTE DÉJÀ DANS train_df
last_train_date = train_df['date'].max()
validation_start_date = last_train_date - pd.DateOffset(days=15)

# On crée le masque DIRECTEMENT à partir de train_df. Les longueurs correspondront parfaitement.
valid_indices = train_df[train_df['date'] >= validation_start_date].index
train_indices = train_df[train_df['date'] < validation_start_date].index

# Créer les jeux de données partiels
train_part_df = train_df.loc[train_indices]
valid_df = train_df.loc[valid_indices]

print(f"Jeu d'entraînement partiel : {train_part_df.shape[0]} lignes")
print(f"Jeu de validation : {valid_df.shape[0]} lignes")

# ==============================================================================
# ÉTAPE 4: DÉFINITION DES FEATURES (X) ET DE LA CIBLE (y)
# ==============================================================================

# La cible est la colonne 'sales'
y_train_part = np.log1p(train_part_df['sales'])
y_valid = np.log1p(valid_df['sales'])

# Les features sont toutes les autres colonnes utiles
# !! CORRECTION : On supprime 'date' et les autres colonnes inutiles ICI !!
features = [col for col in train_df.columns if col not in ['id', 'sales', 'is_train', 'date']]

X_train_part = train_part_df[features]
X_valid = valid_df[features]
X_test = test_df[features] # On utilise la même liste de features pour le test

# Garder les IDs pour la soumission
test_ids = test_df['id'].copy()

# Libérer de la mémoire
del train_df, test_df, data, train_part_df, valid_df
gc.collect()

# ==============================================================================
# ÉTAPE 5: ENTRAÎNEMENT DU MODÈLE LIGHTGBM (inchangé)
# ==============================================================================
print("\nEntraînement du modèle LightGBM avec suivi de la progression...")

lgb_params = {
    'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000,
    'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
    'bagging_freq': 1, 'lambda_l1': 0.1, 'lambda_l2': 0.1, 'num_leaves': 31,
    'verbose': -1, 'n_jobs': -1, 'seed': 42, 'boosting_type': 'gbdt',
}

model = lgb.LGBMRegressor(**lgb_params)

model.fit(X_train_part, y_train_part,
          eval_set=[(X_valid, y_valid)],
          eval_metric='rmse',
          callbacks=[
              lgb.log_evaluation(period=100),
              lgb.early_stopping(stopping_rounds=100)
          ],
          categorical_feature=[col for col in categorical_features if col in features])

# ==============================================================================
# ÉTAPE 6: PRÉDICTION ET CRÉATION DU FICHIER DE SOUMISSION (inchangé)
# ==============================================================================
print("\nGénération des prédictions sur le jeu de test...")

best_iteration = model.best_iteration_ if model.best_iteration_ else lgb_params['n_estimators']
print(f"Meilleure itération trouvée : {best_iteration}")

predictions_log = model.predict(X_test, num_iteration=best_iteration)
predictions = np.expm1(predictions_log)
predictions[predictions < 0] = 0

submission_df = pd.DataFrame({'id': test_ids, 'sales': predictions})
submission_df.to_csv('submission_baseline_lgbm_v2.csv', index=False)

print("\nFichier 'submission_baseline_lgbm_v2.csv' créé avec succès !")
print(submission_df.head())